In [1]:
import os
import json
from tokenizers import Tokenizer,models,pre_tokenizers,trainers

In [2]:
# 加载数据
def load_date(path):
    with open(path, 'r', encoding='utf-8') as f: 
        for line in f:
            data=json.loads(line) #逐行读取，解析line为json
            yield data['text'] # 返回json中的text, yield用于逐步从文件中提取 'text' 字段, yield方法非常适合处理大文件，因为它不会一次性将整个文件加载到内存中，而是逐行处理


## BEP（Byte Pair Encoding）编码
### BPE 的核心思想
BPE 是一种数据压缩算法，后来被应用于自然语言处理中的分词任务。它的核心思想是通过逐步合并频繁出现的字节对（或字符对）来构建词汇表。

### BPE 的工作流程
1. 初始化词汇表:
将文本拆分为最小的单元（如字符或字节）。
例如，文本 `"low lower lowest"` 会被拆分为：`['l', 'o', 'w', ' ', 'l', 'o', 'w', 'e', 'r', ' ', 'l', 'o', 'w', 'e', 's', 't']`。

2. 统计字节对频率:
统计所有相邻字节对的出现频率。
例如，`('l', 'o')` 出现了 3 次，`('o', 'w')` 出现了 3 次。

3. 合并最频繁的字节对:
将最频繁的字节对合并为一个新的符号，并更新词汇表。
例如，合并 `('l', 'o')` 为 `'lo'`，词汇表变为：`['lo', 'w', ' ', 'lo', 'w', 'e', 'r', ' ', 'lo', 'w', 'e', 's', 't']`。

4. 重复合并过程:
重复上述步骤，直到达到预定的词汇表大小或没有更多可合并的字节对。
例如，合并 `('lo', 'w')` 为 `'low'`，词汇表变为：`['low', ' ', 'low', 'e', 'r', ' ', 'low', 'e', 's', 't']`。

5. 最终词汇表:
最终词汇表包含原始字符和合并后的子词。
例如，`['low', ' ', 'lower', ' ', 'lowest']`。

### BPE 的优点
1. 处理未登录词:
通过将未知词拆分为已知的子词，BPE 能够有效地处理未登录词。
例如，`"unhappiness"` 可以被拆分为 `"un", "happiness"`。

2. 词汇表大小可控:
通过调整合并次数，可以控制词汇表的大小。

3. 多语言支持:
BPE 基于字节或字符，因此可以处理多种语言的文本。

In [3]:
tokenizer=Tokenizer(models.BPE()) 
tokenizer.pre_tokenizer=pre_tokenizers.ByteLevel(add_prefix_space=True)